# Object Localization Demonstration Notebook
### Import statements

In [1]:
import pyrealsense2 as rs
import cv2
import detect
import numpy as np
import time
import torch
import io

### Declaring constant variables, obtained from values in Intel and VRC documentation

In [2]:
focal_length = ((448.0-172.0) * 24.0) / 11.0
# ['Blue Goal', 'Blue Platform', 'Blue Robot', 'Neutral Goal', 'Red Goal', 'Red Platform', 'Red Robot', 'Ring']
width_dict = {"7":3.5, "3":12.5, "0":12.5, "4":12.5, "2":5.5, "5":5.5, "1":53.0, "6":53.0}

### Starting image pipeline from D435 camera

In [3]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)

### Object localization method

In [4]:
def obj_distance(obj, depth_frame):
    # file parsing
    obj_array = obj.split()
    x, y, w, h = int(round(float(obj_array[1]))), int(round(float(obj_array[2]))), float(obj_array[3]), float(obj_array[4])

    # calculating distance using trigonometric properties
    trig_distance = (width_dict[str(obj_array[0])] * focal_length)/w
    
    # extract average distance from depth map and convert to inches
    depth_distance_meters = (depth_frame.get_distance(x, y) +\
                             depth_frame.get_distance(x+2, y) +\
                             depth_frame.get_distance(x, y+2) +\
                             depth_frame.get_distance(x-2, y) +\
                             depth_frame.get_distance(x, y-2))/5.0
    depth_distance = 39.3701 * depth_distance_meters
    
    # weighting and combining localization methods
    distance = (trig_distance * .2) + (depth_distance_meters * .8) 
    
    # in the event that depthmap can't detect distance, only use trig distance
    if (depth_distance_meters == 0):
        distance = trig_distance
    
    return distance

### Using identification and localization to process images from the pipeline

In [5]:
while True:
    # extracting data from the image pipeline
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())

    # IO for trained YOLOv5 model (color_img->game_objects)
    cv2.imwrite('img.png', color_image)

    !python detect.py --source img.png --weights static/best_torchscript.pt --conf 0.60 --name yolo_obj --view-img --save-txt --nosave --exist-ok

    # reading labels from localization output
    game_objects = []
    with open('runs/detect/yolo_obj/labels/img.txt', 'r+') as f:
        game_objects = f.readlines()
        f.truncate(0)
        f.close()

    # calculating distance for all game objects in frame
    for obj in game_objects:
        print(f'{obj.split()[0]}: {obj_distance(obj, depth_frame)}')

3: 112.34735413839891
3: 119.48051948051948


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  a0dc65c torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 2 class3s, Done. (1.435s)
Speed: 2.0ms pre-process, 1434.8ms inference, 5.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
1 labels saved to runs\detect\yolo_obj\labels


7: 58.54545454545454
7: 95.80165289256198
7: 20.498167288520122
7: 67.9882697947214
7: 9.429305407083241
7: 55.464114832535884
7: 95.80165289256198
7: 49.014799154334035
3: 34.371108343711086
7: 67.9882697947214
7: 87.81818181818181
3: 61.69895678092399
7: 84.30545454545454
7: 63.86776859504132
7: 16.936770912257106
7: 11.66530298703025
3: 76.80890538033395
7: 8.229580637035948
7: 50.18181818181818
7: 47.90082644628099


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  a0dc65c torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 3 class3s, 17 class7s, Done. (1.063s)
Speed: 3.0ms pre-process, 1063.3ms inference, 2.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
1 labels saved to runs\detect\yolo_obj\labels


7: 11.86557091626254
7: 13.755253968385658
7: 20.234327278353952
3: 17.872029950529495
7: 9.938479838477129
3: 10.25812178453052
7: 12.204436367641796
7: 14.552509108456698
7: 9.796872746727683


...

7: 63.86776859504132
7: 61.98930481283422
3: 203.43980343980343
7: 7.524175319626001
7: 36.338557993730404
7: 31.457259158751693
7: 31.457259158751693


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  a0dc65c torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 1 class3, 6 class7s, Done. (1.469s)
Speed: 3.0ms pre-process, 1469.4ms inference, 6.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
1 labels saved to runs\detect\yolo_obj\labels
detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=F

7: 9.369876373897897
7: 8.47644064056682
7: 8.642705464883283
7: 8.47548065292644
7: 8.015020614681822
7: 8.33702478781777
7: 7.563410060644517
7: 8.104612244298636
3: 11.910030286438118


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  a0dc65c torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 1 class3, 8 class7s, Done. (1.853s)
Speed: 4.0ms pre-process, 1852.9ms inference, 6.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
1 labels saved to runs\detect\yolo_obj\labels
detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=F

RuntimeError: Frame didn't arrive within 5000